In [1]:
!pip install openpyxl


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import re
from nltk.tokenize import TweetTokenizer

In [3]:
df = pd.read_excel("./Raw-Dataset/riberiioo_tweets.xlsx")
df.head()

,CREATED_AT,TEXT,USER_NAME,USER_SCREEN_NAME,DETECTED_SOURCE_LANGUAGE,USER_COUNTRY_DETECTION,USER_FOLLOWERS_COUNT,USER_FRIENDS_COUNT,RETWEET_COUNT,QUOTE_COUNT,TWEETLINK
0,"24/11/2022 13:44:24,000000000",@superlig @Besiktas Neden 8 abi neden. Yapıştı...,Riberio,riberiioo,tr,UNKNOWN,9985,3209,0,2,https://twitter.com/riberiioo/status/159577534...
1,"29/11/2022 21:08:11,000000000",Sizce Tareminin son dakikada ki pozisyonu pena...,Riberio,riberiioo,tr,UNKNOWN,9985,3209,0,0,https://twitter.com/riberiioo/status/159769896...
2,"25/11/2022 20:42:05,000000000",Anti İngiltereciler beğensin. Sayımızı bilelim...,Riberio,riberiioo,tr,UNKNOWN,9985,3209,0,3,https://twitter.com/riberiioo/status/159624284...
3,"28/11/2022 20:54:26,000000000",Torreirasız maç kazanmayı bekliyor adam. Bu ma...,Riberio,riberiioo,tr,UNKNOWN,9985,3209,0,0,https://twitter.com/riberiioo/status/159733311...
4,"05/12/2022 13:44:16,000000000",@ukarakullukcu Bu adam kim?,Riberio,riberiioo,tr,UNKNOWN,9985,3209,0,0,https://twitter.com/riberiioo/status/159976157...


In [4]:
print(f"Initial dataset shape: {df.shape}")

Initial dataset shape: (11197, 11)


In [5]:
df.isnull().sum()

CREATED_AT                    0
TEXT                          0
USER_NAME                     0
USER_SCREEN_NAME              0
DETECTED_SOURCE_LANGUAGE    254
USER_COUNTRY_DETECTION        0
USER_FOLLOWERS_COUNT          0
USER_FRIENDS_COUNT            0
RETWEET_COUNT                 0
QUOTE_COUNT                   0
TWEETLINK                     0
dtype: int64

In [ ]:
df = df.dropna(subset=['DETECTED_SOURCE_LANGUAGE'])

In [ ]:
df = df[df['DETECTED_SOURCE_LANGUAGE'] == 'tr']

In [ ]:
print(f"Dataset shape after filtering for Turkish language: {df.shape}")

Dataset shape after filtering for Turkish language: (10648, 11)


In [ ]:
df = df.drop(columns=['USER_COUNTRY_DETECTION'])
df = df.drop(columns=['USER_SCREEN_NAME'])

In [ ]:
# Update the cleaning function
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove mentions (@username)
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags (#hashtag)
    text = re.sub(r'#\w+', '', text)
    # Remove punctuation and numbers, keep Turkish characters
    text = re.sub(r'[^a-zA-ZçÇğĞıİöÖşŞüÜ\s]', '', text)
    # Remove additional symbols {, [, ;, ', ", :
    text = re.sub(r'[\{\[\;\'\"\}:]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [ ]:
df['CLEANED_TEXT'] = df['TEXT'].apply(clean_text)

df[['TEXT', 'CLEANED_TEXT']].head()

,TEXT,CLEANED_TEXT
0,@superlig @Besiktas Neden 8 abi neden. Yapıştı...,neden abi neden yapıştı üzerimize ten beri beş...
1,Sizce Tareminin son dakikada ki pozisyonu pena...,sizce tareminin son dakikada ki pozisyonu pena...
2,Anti İngiltereciler beğensin. Sayımızı bilelim...,anti i̇ngiltereciler beğensin sayımızı bilelim
3,Torreirasız maç kazanmayı bekliyor adam. Bu ma...,torreirasız maç kazanmayı bekliyor adam bu maç...
4,@ukarakullukcu Bu adam kim?,bu adam kim


In [13]:
df = df[df['CLEANED_TEXT'].str.strip() != '']

In [14]:
df['CLEANED_TEXT'] = df['CLEANED_TEXT'].apply(lambda x: x.encode('utf-8').decode('utf-8'))


C:\Users\talha\AppData\Local\Temp\ipykernel_16080\3621796786.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CLEANED_TEXT'] = df['CLEANED_TEXT'].apply(lambda x: x.encode('utf-8').decode('utf-8'))


In [ ]:
def lowercase_turkish(text):
    mapping = str.maketrans('Iİ', 'ıi')
    return text.translate(mapping).lower()

df['CLEANED_TEXT'] = df['CLEANED_TEXT'].apply(lowercase_turkish)


C:\Users\talha\AppData\Local\Temp\ipykernel_16080\412586288.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CLEANED_TEXT'] = df['CLEANED_TEXT'].apply(lowercase_turkish)


In [ ]:
from collections import Counter

tokenizer = TweetTokenizer()

all_words = []
for text in df['CLEANED_TEXT']:
    tokens = tokenizer.tokenize(text)
    all_words.extend(tokens)

word_freq = Counter(all_words)
common_words = word_freq.most_common(50)

print("Most common words:", common_words)

turkish_stop_words = [
    've', 'bir', 'bu', 'için', 'ile', 'daha', 'çok', 'ama', 'gibi', 'de', 'da', 
    'ise', 'çünkü', 'her', 'şey', 'kadar', 'neden', 'nasıl', 'ne', 'kim', 'o', 
    'onu', 'ona', 'ben', 'sen', 'biz', 'siz', 'onlar', 'şu', 'yine', 'hiç', 
    'artık', 'ki', 'mı', 'mi', 'mu', 'mü', 'biraz', 'biri', 'hemen', 'cart', 'curt'
    # Add more Turkish stop words as needed
]

def remove_stop_words(text):
    tokens = tokenizer.tokenize(text)  # Tokenize the text
    filtered_words = [word for word in tokens if word not in turkish_stop_words]
    return ' '.join(filtered_words)

df['CLEANED_TEXT'] = df['CLEANED_TEXT'].apply(remove_stop_words)

Most common words: [('bu', 2752), ('bir', 2556), ('çok', 1121), ('de', 1089), ('en', 1085), ('galatasaray', 1035), ('da', 912), ('var', 821), ('için', 791), ('daha', 776), ('o', 728), ('ne', 720), ('yok', 697), ('ve', 696), ('gol', 667), ('maç', 649), ('diye', 642), ('sonra', 636), ('kadar', 590), ('penaltı', 545), ('ama', 542), ('fenerbahçe', 509), ('sen', 507), ('her', 502), ('değil', 487), ('gibi', 482), ('şampiyon', 464), ('icardi', 457), ('ki', 456), ('mı', 435), ('şu', 428), ('ya', 426), ('bile', 425), ('iyi', 417), ('biz', 394), ('sezon', 377), ('maçı', 377), ('son', 375), ('büyük', 375), ('mi', 374), ('hakem', 341), ('nasıl', 339), ('adam', 334), ('puan', 329), ('sene', 325), ('takım', 318), ('yine', 304), ('böyle', 301), ('ali', 297), ('siz', 289)]


C:\Users\talha\AppData\Local\Temp\ipykernel_16080\995217177.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CLEANED_TEXT'] = df['CLEANED_TEXT'].apply(remove_stop_words)


In [ ]:
from nltk import ngrams

bigrams = list(ngrams(all_words, 2))
bigram_freq = Counter(bigrams).most_common(20)
print("Most common bigrams:", bigram_freq)

trigrams = list(ngrams(all_words, 3))
trigram_freq = Counter(trigrams).most_common(20)
print("Most common trigrams:", trigram_freq)


Most common bigrams: [(('bu', 'kadar'), 219), (('en', 'iyi'), 217), (('ali', 'koç'), 142), (('bir', 'de'), 139), (('mauro', 'icardi'), 130), (('en', 'çok'), 130), (('kırmızı', 'kart'), 120), (('barış', 'alper'), 115), (('bu', 'sezon'), 113), (('en', 'büyük'), 110), (('hem', 'de'), 108), (('bu', 'sene'), 103), (('o', 'sırada'), 100), (('böyle', 'bir'), 98), (('okan', 'buruk'), 97), (('kerem', 'aktürkoğlu'), 96), (('geçen', 'sezon'), 91), (('bir', 'daha'), 90), (('hakim', 'ziyech'), 90), (('cart', 'curt'), 88)]
Most common trigrams: [(('barış', 'alper', 'yılmaz'), 64), (('cart', 'curt', 'o'), 61), (('curt', 'o', 'sırada'), 61), (('ligin', 'en', 'iyi'), 38), (('galatasaray', 'tarihinin', 'en'), 29), (('çok', 'büyük', 'bir'), 29), (('halil', 'umut', 'meler'), 27), (('süper', 'kupa', 'maçına'), 27), (('tarihinin', 'en', 'iyi'), 27), (('mert', 'hakan', 'yandaş'), 25), (('köy', 'takımından', 'tane'), 23), (('yatacak', 'yeriniz', 'yok'), 23), (('dünyanın', 'en', 'güzel'), 23), (('fıkra', 'bu',

In [ ]:
df.to_csv('cleaned_tweets.csv', index=False)